In [67]:
import pandas as pd

df_t = pd.read_csv('True.csv', header=0)
df_f = pd.read_csv('Fake.csv', header=0)

print(df_t.shape)
print(df_f.shape)


(21417, 4)
(23481, 4)


In [75]:
# Adding labels
df_t['label'] = 0
df_f['label'] = 1

# Appending data
df = df_t.append(df_f, ignore_index = True)

# Shuffling trainset
from sklearn.utils import shuffle

df = shuffle(df)
print(df.iloc[2,0])

 Obama Singing ‘Happy Birthday’ To Malia Is The Most ‘Dad’ Thing You’ll Ever See Him Do (VIDEO)


C:\Users\RuturajMane\AppData\Local\Temp\ipykernel_3012\683508068.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_t.append(df_f, ignore_index = True)


In [76]:
import tensorflow as tf
print(tf.__version__)

2.9.1


In [94]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense,Dropout

In [78]:
# Vocabulary size 
vocab_size = 5000

In [79]:
import nltk
import re
from nltk.corpus import stopwords

In [80]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RuturajMane\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [81]:
# Data preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(len(df)):
    review = re.sub('[^a-zA-z]', ' ', df.iloc[i,0])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
    corpus.append(' '.join(review))

In [82]:
# One hot representation
onehot_rep = [one_hot(words,vocab_size) for words in corpus]
print(onehot_rep)

[[569, 446, 4466, 1149, 4769], [1582, 3985, 2086, 3952, 3289, 3087, 2432, 3985, 891, 2037, 929], [1281, 2833, 2237, 4997, 4255, 3645, 4201, 503, 4697, 3691], [2072, 260, 3718, 1252, 1168, 1281, 4565, 4424, 1709, 942, 3159, 871, 3970, 4013, 2507, 3398, 3691], [891, 1753, 4622, 115, 2309, 3009, 811], [3593, 3371, 3984, 3009, 202, 4041, 2401, 891, 442], [1429, 2505, 4299, 2786, 1895, 356], [3577, 4299, 3348, 627, 1163, 846, 683, 453, 1280, 953, 2137], [1281, 1814, 891, 1155, 3375, 4299, 1798, 1633], [1748, 4341, 3195, 3916, 3241, 3786, 804, 3332, 3985, 3479, 4936, 3329, 1940, 2930, 1677, 2396, 2342, 1470], [4116, 2371, 2096, 2725, 2830, 3694, 1352, 1678, 3112, 859, 4036], [4113, 244, 2645, 3758, 2146, 2676, 1235, 1469], [3970, 840, 3239, 4616, 2569, 3685, 4951, 175], [3577, 347, 3681, 4431, 2385, 4697, 1071, 3406, 1274, 1838, 4262, 3691], [3922, 1981, 15, 3009, 27, 4815, 3984, 4940], [4855, 904, 2110, 2985, 4299, 4028, 266, 2076, 3666, 4378, 3691], [2669, 4252, 2791, 4939, 3653, 933, 2120

In [83]:
# Pad_sequence is to ensure that the lenght of words in each sentence is same
sent_len = 20
embedded_doc = pad_sequences(onehot_rep, padding = 'pre', maxlen = sent_len) #Pre will add zeros in the preceeding
# pad_sequences's first argument is onehot_rep which is list containing list
print(embedded_doc)

[[   0    0    0 ... 4466 1149 4769]
 [   0    0    0 ...  891 2037  929]
 [   0    0    0 ...  503 4697 3691]
 ...
 [   0    0    0 ...  474  411 1307]
 [   0    0    0 ... 1945 3970 1096]
 [   0    0    0 ...  301 1537 3691]]


In [84]:
len(embedded_doc)

44898

In [85]:
# No. of feature with which each word will be represented
embedding_feature_vector = 50

In [112]:
import keras
opt = keras.optimizers.Adam(learning_rate=1e-06)
# Creating model
model = Sequential()
model.add(Embedding(vocab_size, embedding_feature_vector, input_length = sent_len))
model.add(LSTM(100)) #LSTM with 100 hidden neurons
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer = opt, metrics = ['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 20, 50)            250000    
                                                                 
 lstm_6 (LSTM)               (None, 100)               60400     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________
None


In [113]:
y = df['label']
x = embedded_doc

import numpy as np
x = np.array(x)
y = np.array(y)

In [114]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.33, random_state = 42)
# x_train = x_train.reset_index()
# x_test = x_test.reset_index()
# y_train = y_train.reset_index()
# y_test = y_test.reset_index()

print(x_train)
print(y_train)
print(x_train.shape)
print(y_train.shape)

[[   0    0    0 ... 2120 2663 2390]
 [   0    0    0 ... 1884 4499  928]
 [   0    0    0 ... 3014  949 3691]
 ...
 [   0    0    0 ... 3407 3676 2578]
 [   0    0    0 ... 1898 2435 2202]
 [   0    0    0 ... 1687 1711 3666]]
[0 1 1 ... 0 1 0]
(30081, 20)
(30081,)


In [115]:
# Model training
model.fit(x_train, y_train, epochs=10, batch_size = 64, validation_data = (x_test, y_test))

Epoch 1/10
471/471 [==============================] - 11s 21ms/step - loss: 0.6930 - accuracy: 0.5103 - val_loss: 0.6927 - val_accuracy: 0.5437
Epoch 2/10
471/471 [==============================] - 12s 25ms/step - loss: 0.6925 - accuracy: 0.5689 - val_loss: 0.6922 - val_accuracy: 0.5986
Epoch 3/10
471/471 [==============================] - 13s 28ms/step - loss: 0.6920 - accuracy: 0.6171 - val_loss: 0.6917 - val_accuracy: 0.6428
Epoch 4/10
471/471 [==============================] - 14s 30ms/step - loss: 0.6915 - accuracy: 0.6616 - val_loss: 0.6912 - val_accuracy: 0.6798
Epoch 5/10
471/471 [==============================] - 14s 30ms/step - loss: 0.6910 - accuracy: 0.6928 - val_loss: 0.6907 - val_accuracy: 0.7086
Epoch 6/10
471/471 [==============================] - 14s 30ms/step - loss: 0.6904 - accuracy: 0.7165 - val_loss: 0.6901 - val_accuracy: 0.7282
Epoch 7/10
471/471 [==============================] - 15s 31ms/step - loss: 0.6897 - accuracy: 0.7317 - val_loss: 0.6894 - val_accuracy:

In [116]:
# Accuracy and confusin matrix

predict_x=model.predict(x_test) 
classes_x=np.argmax(predict_x,axis=1)

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, classes_x)

464/464 [==============================] - 3s 5ms/step


array([[7096,    0],
       [7721,    0]], dtype=int64)

In [117]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, classes_x)

0.47890936086927177